**Importing the dependencies**

In [3]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


2025-09-10 15:53:29.640421: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-10 15:53:29.770957: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-10 15:53:32.836702: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Collect data using kaggle API

In [4]:
kaggle_dictionary = json.load(open('../kaggle.json'))

In [5]:
# setup kaggle API
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_kEY'] = kaggle_dictionary['key']

In [6]:
#https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
with ZipFile('../archive.zip','r') as zip_ref:
    zip_ref.extractall('../data')

Load the data into a pandas dataframe

In [7]:
data = pd.read_csv('../data/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Check the sentiment column for consistent  data

In [8]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

Set positive to 1 and negative to 0

In [9]:
data['sentiment'].replace({'positive':1, 'negative':0}, inplace=True)
data['sentiment'].value_counts()

/tmp/ipykernel_121577/1588877590.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['sentiment'].replace({'positive':1, 'negative':0}, inplace=True)
/tmp/ipykernel_121577/1588877590.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['sentiment'].replace({'positive':1, 'negative':0}, inpl

sentiment
1    25000
0    25000
Name: count, dtype: int64

Split the data into train and test data

In [10]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print(train_data.shape); print(test_data.shape)

(40000, 2)
(10000, 2)


Data Processing

In [ ]:
tokenizer = Tokenizer(num_words = 50000)
tokenizer.fit_on_texts(train_data['review'])
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen = 200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen = 200)
y_train = train_data['sentiment']
y_test = test_data['sentiment']
y_train  



39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64

Build the model with Long Short Term Memory (LSTM)

In [22]:
# export the tokenizer to use in the streamlit app
import joblib
joblib.dump(tokenizer, '../model/tokenizer.pkl')

['../model/tokenizer.pkl']

In [12]:
model = Sequential()
model.add(Embedding(input_dim=50000, output_dim=128, input_length = 200))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.build(input_shape=(None, 200))


model.summary()

/media/neba/New Volume/Internship/venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-09-10 15:54:05.738878: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │     6,400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,531,713 (24.92 MB)

 Trainable params: 6,531,713 (24.92 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
#compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


**Training the model**

In [24]:
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2 )

Epoch 1/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 351s 348ms/step - accuracy: 0.9401 - loss: 0.1570 - val_accuracy: 0.8441 - val_loss: 0.4452
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 320s 320ms/step - accuracy: 0.9684 - loss: 0.0905 - val_accuracy: 0.8708 - val_loss: 0.4444
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 352s 352ms/step - accuracy: 0.9836 - loss: 0.0500 - val_accuracy: 0.8656 - val_loss: 0.4858
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 336s 336ms/step - accuracy: 0.9905 - loss: 0.0311 - val_accuracy: 0.8673 - val_loss: 0.5831
Epoch 5/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 328s 328ms/step - accuracy: 0.9912 - loss: 0.0273 - val_accuracy: 0.8708 - val_loss: 0.6075


In [25]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 46ms/step - accuracy: 0.8714 - loss: 0.5854
Test Loss: 0.5853869915008545
Test Accuracy: 0.871399998664856


Build a predictive system

In [26]:
def predict_sentiment(review):
    #tokenize and pad the review
    sequence = tokenizer.texts_to_sequences(review)
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'
    return sentiment

In [43]:
 # example usage
review = "wow"
sentiment = predict_sentiment(review)
print(f'the sentiment is {sentiment}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
the sentiment is positive


In [ ]:
#export the model using joblib
import joblib
joblib.dump(model, '../model/model.pkl')

['../model/model.pkl']

: 

: 